# Demo Model

In [73]:
import pandas as pd
import numpy as np
import statistics as st
from numpy.random import normal

# Preparing Dataset
 Taking necessary features


In [23]:
players_gl = ['AJ Finch','BB McCullum','SK Raina','KD Karthik','DJ Bravo','RA Jadeja','AD Nath','DW Steyn','P Kumar']
players_srh = ['DA Warner','S Dhawan']

In [24]:
predict_id = 1260

In [25]:
previd_gl = [1242,1248]

In [26]:
previd_srh = [1244,1252]

In [27]:
gl_pd = pd.DataFrame()
srh_pd = pd.DataFrame()


In [28]:
def prepare_data(mid,df,name):
    
    ds = []
    
    ds.append(mid)
    ds.append(name)
    
    cols = ['runs','balls','strike_rate','fours','sixes']
    
    if (df.match==mid).any():
        dff = df[df.match==mid]
        for i in cols:
            ds.append(int(dff[i]) )
    else:
        for i in cols:
            ds.append(0)
    
    fds = pd.DataFrame(data=[ds],columns=['match_id','name','runs','balls','strike_rate','fours','sixes'])
    
    return fds

In [29]:
Run=1
Boundary=1
Six=2
Half=8
Century=16
Duck=-2

In [43]:
loc = r"D:\arbunize\ipl_dataset\batsmen_ipl\\"

In [44]:
players = {}

In [46]:
for name in players_gl:
    
    df = pd.DataFrame(columns=['match_id','name','runs','balls','strike_rate','fours','sixes'])
    
    for mid in previd_gl:
        
        floc = loc    + name +".csv"
        #rfloc = r'{}'.format(floc)
        d = pd.read_csv(floc)
        #print(d['match'])
        print(name,mid)
        df = df.append( prepare_data(mid,d,name) , ignore_index=True )
     
    
    players[name] = df
    
        
        
        
        

AJ Finch 1242
AJ Finch 1248
BB McCullum 1242
BB McCullum 1248
SK Raina 1242
SK Raina 1248
KD Karthik 1242
KD Karthik 1248
DJ Bravo 1242
DJ Bravo 1248
RA Jadeja 1242
RA Jadeja 1248
AD Nath 1242
AD Nath 1248
DW Steyn 1242
DW Steyn 1248
P Kumar 1242
P Kumar 1248


In [47]:
for i in players.keys():
    print(i)

AJ Finch
BB McCullum
SK Raina
KD Karthik
DJ Bravo
RA Jadeja
AD Nath
DW Steyn
P Kumar


In [48]:
for name in players_srh:
    
    df = pd.DataFrame(columns=['match_id','name','runs','balls','strike_rate','fours','sixes'])
    
    for mid in previd_srh:
        
        floc = loc   + name +".csv"
        #rfloc = r'{}'.format(floc)
        d = pd.read_csv(floc)
        #print(d['match'])
        print(name,mid)
        df = df.append( prepare_data(mid,d,name) , ignore_index=True )
     
    
    players[name] = df

DA Warner 1244
DA Warner 1252
S Dhawan 1244
S Dhawan 1252


In [49]:
for i in players.keys():
    print(i)

AJ Finch
BB McCullum
SK Raina
KD Karthik
DJ Bravo
RA Jadeja
AD Nath
DW Steyn
P Kumar
DA Warner
S Dhawan


# Fantasy Pts Calculation
## From necessary features

In [98]:
def calc_fantasy_pts(players,n):
    fpts_dict = {}
    for i in players.values():
        fp=[]
        for j in range(n):
            
            fpts =0 
            fpts+=int(i['runs'][j])
            fpts+=int(i['fours'][j])
            
            fpts+=2*int(i['sixes'][j])
            
            if int(i['runs'][j])>=50:
                fpts+=8
            if int(i['runs'][j])>=100:
                fpts+=16
            if int(i['runs'][j])==0 and int(i['balls'][j])!=0:
                fpts-=2
            fp.append(fpts)
            
        fpts_dict[str(i['name'][0])] = fp
    return fpts_dict
        
            
    

In [99]:
fpts_dict = calc_fantasy_pts(players,2)

In [100]:
for i in fpts_dict.values():
    print(i)

[94, 69]
[-2, 58]
[25, 25]
[48, 0]
[2, 27]
[8, 4]
[0, 0]
[0, 0]
[0, 0]
[80, 15]
[9, 7]


# Monte Carlo Simulations For Calculating Multiple Fantasy Pts

In [101]:
def simulation_pts(fpts_dict):
    sim_pts = {}
    p = list(fpts_dict.keys())
    i=0
    for j in fpts_dict.values():
        mu = st.mean(j)
        sig = st.stdev(j)
        fpts = normal(mu,sig,100)
        sim_pts[p[i]] = fpts
        i+=1
    return sim_pts
        

In [102]:
sim_pts = simulation_pts(fpts_dict)

In [116]:
#for i in sim_pts.values():
    #print(i[0])

In [123]:
plyr = list(sim_pts.keys())


# Optimizing the Result to obtain Probabilities

In [105]:
def optimize(sim_pts):
    ind=[]
    for i in range(100):
        pts = []
        for j in sim_pts.values():
            pts.append(j[i])
        pts = np.array(pts)
        ind.append( pts.argsort()[-3:][::-1] )
    return ind    
        
        

In [106]:
ind = optimize(sim_pts)

In [107]:
str(list(ind[0]))

'[0, 1, 2]'

In [108]:
dict_ind ={}
for i in ind:
    if str(list(i)) in dict_ind.keys():
        dict_ind[str(list(i) )]+=1
    else:
        dict_ind[str(list(i))]=1

In [109]:
dict_ind

{'[0, 1, 2]': 6,
 '[0, 9, 2]': 11,
 '[9, 0, 3]': 13,
 '[0, 9, 4]': 3,
 '[0, 1, 9]': 9,
 '[0, 4, 2]': 4,
 '[0, 3, 1]': 6,
 '[0, 9, 3]': 4,
 '[3, 9, 0]': 1,
 '[3, 1, 0]': 1,
 '[0, 3, 4]': 2,
 '[0, 1, 4]': 1,
 '[1, 0, 3]': 5,
 '[0, 2, 1]': 1,
 '[3, 0, 1]': 1,
 '[0, 1, 3]': 4,
 '[0, 2, 4]': 1,
 '[0, 3, 9]': 1,
 '[9, 0, 1]': 3,
 '[0, 9, 1]': 5,
 '[9, 0, 2]': 5,
 '[0, 3, 2]': 2,
 '[9, 3, 1]': 1,
 '[0, 2, 3]': 2,
 '[3, 0, 2]': 2,
 '[1, 0, 9]': 2,
 '[0, 2, 10]': 1,
 '[1, 0, 2]': 1,
 '[9, 0, 4]': 1,
 '[9, 3, 0]': 1}

# Name of predicted players

In [117]:
# RESULT
print(plyr[0],plyr[9],plyr[2])

AJ Finch DA Warner SK Raina


# Checking Actual Results

In [112]:
fplyr={}
for name in plyr:
    
    df = pd.DataFrame(columns=['match_id','name','runs','balls','strike_rate','fours','sixes'])
    
    for mid in [predict_id]:
        
        floc = loc    + name +".csv"
        #rfloc = r'{}'.format(floc)
        d = pd.read_csv(floc)
        #print(d['match'])
        print(name,mid)
        df = df.append( prepare_data(mid,d,name) , ignore_index=True )
     
    
    fplyr[name] = df
    

AJ Finch 1260
BB McCullum 1260
SK Raina 1260
KD Karthik 1260
DJ Bravo 1260
RA Jadeja 1260
AD Nath 1260
DW Steyn 1260
P Kumar 1260
DA Warner 1260
S Dhawan 1260


In [114]:
fpts = calc_fantasy_pts(fplyr,1)

In [115]:
fpts

{'AJ Finch': [-2],
 'BB McCullum': [21],
 'SK Raina': [92],
 'KD Karthik': [9],
 'DJ Bravo': [8],
 'RA Jadeja': [14],
 'AD Nath': [5],
 'DW Steyn': [1],
 'P Kumar': [1],
 'DA Warner': [91],
 'S Dhawan': [66]}

In [119]:
# Final Result
[2,9,10]

[2, 9, 10]

In [120]:
#Predicted Result
[0,2,9]

[0, 2, 9]

# Two Correctly Predicted